In [2]:
import os
import re
import string
import pandas as pd
import urllib.parse

In [3]:
# Configuración de rutas
core_data_path = "core_data"
output_path = "docs"

# Leer los archivos TSV
colaboradores_tsv = os.path.join(core_data_path, "Diccionario_angl_tec_mx_private - Colaboradores.tsv")
terminos_tsv = os.path.join(core_data_path, "Diccionario_angl_tec_mx_private - Términos.tsv")

In [4]:
colaboradores_df = pd.read_csv(colaboradores_tsv, sep='\t')
terminos_df = pd.read_csv(terminos_tsv, sep='\t')

In [16]:
# Función para limpiar y generar nombres de archivo seguros
def generar_nombre_archivo(termino):
    return re.sub(r'[^a-zA-Z0-9\-]', '', termino.lower().replace(' ', '-'))

# Función para crear hipervínculos de términos relacionados
def generar_hipervinculos(terminos, tipo):
    links = []
    for termino in terminos:
        safe_name = generar_nombre_archivo(termino)
        if tipo == "alfabeticamente":
            letra_inicial = termino[0].upper()
            link = f"[{termino}](https://maleniski.github.io/diccionario-angl-tec-mx/docs/alfabeticamente/{letra_inicial}/{safe_name}.html)"
        elif tipo == "taxonomia":
            taxonomia_name = generar_nombre_archivo(termino)
            link = f"[{termino}](https://maleniski.github.io/diccionario-angl-tec-mx/docs/taxonomia/{taxonomia_name}/{safe_name}.html)"
        links.append(link)
    return ', '.join(links)

# Crear archivo index.md en la raíz de "alfabeticamente" y "taxonomia"
def crear_index_general(ruta, titulo, orden):
    index_md = f"""---
layout: default
title: {titulo}
nav_order: {orden}
has_children: true
---"""
    with open(os.path.join(ruta, 'index.md'), 'w') as f:
        f.write(index_md)

# Crear directorios principales
alfabeticamente_path = os.path.join(output_path, "alfabeticamente")
taxonomia_path = os.path.join(output_path, "taxonomia")

os.makedirs(alfabeticamente_path, exist_ok=True)
os.makedirs(taxonomia_path, exist_ok=True)

# Crear los archivos index.md en las carpetas principales con nav_order ajustado
crear_index_general(alfabeticamente_path, "Alfabéticamente", 2)
crear_index_general(taxonomia_path, "Taxonomía", 3)

# Inicializar contadores de nav_order para cada carpeta
nav_order_alfabetico = {}
nav_order_taxonomia = {}

# Crear directorios y archivos Markdown
for _, row in terminos_df.iterrows():
    termino = row['Término']
    definicion = row['Definición']
    taxonomia = row['Taxonomía']
    relacionados = row['Relacionados'].split(', ') if pd.notna(row['Relacionados']) else []

    # Determinar carpeta alfabética
    letra_inicial = termino[0].upper()
    alphabet_folder = os.path.join(alfabeticamente_path, letra_inicial)
    termino_folder = os.path.join(alphabet_folder, generar_nombre_archivo(termino))

    # Crear carpeta alfabética si no existe
    os.makedirs(alphabet_folder, exist_ok=True)

    if letra_inicial not in nav_order_alfabetico:
        nav_order_alfabetico[letra_inicial] = len(nav_order_alfabetico) + 1
        # Crear un archivo index.md para la letra con has_children: true
        index_md = f"""---
layout: default
title: {letra_inicial}
nav_order: {nav_order_alfabetico[letra_inicial]}
parent: Alfabéticamente
has_children: true
---"""
        with open(os.path.join(alphabet_folder, 'index.md'), 'w') as f:
            f.write(index_md)

    # Crear contenido del archivo Markdown para el término en la carpeta alfabética
    contenido_md_alfabetico = f"""---
layout: default
title: {termino}
nav_order: {nav_order_alfabetico[letra_inicial] + len(os.listdir(alphabet_folder)) - 1}
parent: {letra_inicial}
grand_parent: Alfabéticamente
---

# {termino}

**Definición:**  
{definicion}

**Taxonomía:**  
{taxonomia}

**Términos relacionados:**  
{generar_hipervinculos(relacionados, 'alfabeticamente')}
"""

    # Guardar archivo Markdown en la carpeta correspondiente
    with open(os.path.join(alphabet_folder, f'{generar_nombre_archivo(termino)}.md'), 'w') as f:
        f.write(contenido_md_alfabetico)
    
    # Para taxonomía, un enfoque similar
    taxonomy_folder = os.path.join(taxonomia_path, generar_nombre_archivo(taxonomia))
    termino_folder_taxonomia = os.path.join(taxonomy_folder, generar_nombre_archivo(termino))

    # Crear carpeta de taxonomía si no existe
    os.makedirs(taxonomy_folder, exist_ok=True)

    if taxonomia not in nav_order_taxonomia:
        nav_order_taxonomia[taxonomia] = len(nav_order_taxonomia) + 1
        # Crear un archivo index.md para la taxonomía con has_children: true
        index_md = f"""---
layout: default
title: {taxonomia}
nav_order: {nav_order_taxonomia[taxonomia]}
parent: Taxonomía
has_children: true
---"""
        with open(os.path.join(taxonomy_folder, 'index.md'), 'w') as f:
            f.write(index_md)

    contenido_md_taxonomia = f"""---
layout: default
title: {termino}
nav_order: {nav_order_taxonomia[taxonomia] + len(os.listdir(taxonomy_folder)) - 1}
parent: {taxonomia}
grand_parent: Taxonomía
---

# {termino}

**Definición:**  
{definicion}

**Taxonomía:**  
{taxonomia}

**Términos relacionados:**  
{generar_hipervinculos(relacionados, 'taxonomia')}
"""

    # Guardar archivo Markdown en la carpeta correspondiente
    with open(os.path.join(taxonomy_folder, f'{generar_nombre_archivo(termino)}.md'), 'w') as f:
        f.write(contenido_md_taxonomia)